In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from Bio import AlignIO
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from scipy.stats import fisher_exact

In [3]:
homoplasy_dominant_clust_enrich=pickle.load(open( "../vars/homoplasy_dominant_cluster_enrichment", "rb" ) )
homoplasy_dominant_clust_enrich_top=homoplasy_dominant_clust_enrich[homoplasy_dominant_clust_enrich.homoplasy_count>=2].sort_values(by='enrichment_p', ascending=True).iloc[0:50,:]

In [4]:
pd.set_option('display.max_colwidth', None)
homoplasy_dominant_clust_enrich_top.reset_index()

,index,Position,major,minor,Gene,homoplasy_count,Type,Product,enrichment_p,OR
0,53445,4770807,G,A,MAB_4690c,2,intragenic,Linear gramicidin synthase subunit D,1.923486e-71,inf
1,47313,4211627,C,T,MAB_4147c,2,intragenic,hypothetical protein,3.635388e-69,inf
2,13841,1217813,C,T,MAB_1203,3,intragenic,Mycothiol S-conjugate amidase,4.254548e-67,2914.000000
3,38491,3381893,C,G,MAB_3334c,4,intragenic,Aspartyl/glutamyl-tRNA(Asn/Gln) amidotransferase subunit B,1.034483e-65,inf
4,24054,2119889,G,T,MAB_2120c,6,intragenic,Putative inactive phenolphthiocerol synthesis polyketide synthase type I Pks15,1.709089e-64,inf
5,37464,3279849,G,A,MAB_3243,4,intragenic,Soluble secreted antigen MPT53,1.716140e-64,inf
6,37476,3280500,G,A,MAB_3244,4,intragenic,hypothetical protein,1.716140e-64,inf
7,37480,3280832,G,A,MAB_3244,4,intragenic,hypothetical protein,1.716140e-64,inf
8,39890,3520983,G,T,MAB_3480,6,intragenic,hypothetical protein,7.209222e-64,inf
9,34088,3016615,G,A,MAB_2961,3,intragenic,"2-methoxy-6-polyprenyl-1,4-benzoquinol methylase, mitochondrial",7.750096e-63,inf


In [5]:
pos_gene_dict=homoplasy_dominant_clust_enrich_top[['Position', 'Gene']].set_index('Position').to_dict('index')
pos_gene_dict['1217813']['Gene']

'MAB_1203'

In [6]:
snp_pos=[line.rstrip('\n') for line in open('/n/data1/hms/dbmi/farhat/nikki/abscessus/0_NOTEBOOKS/010_homoplasy/vars/mab_masked_snp_positions.txt')]
snp_pos_series=pd.Series(snp_pos)
snp_pos_dict=dict(zip(snp_pos_series, snp_pos_series.index))

masked_snpAln_unwrapped="/n/data1/hms/dbmi/farhat/nikki/abscessus/0_NOTEBOOKS/010_homoplasy/vars/mab_masked_snpAln_unwrapped.fasta"
aln_masked=AlignIO.read(masked_snpAln_unwrapped, "fasta")

In [7]:
allele_dict={}
gene_list=[]
allele_dict['name']=[i.id for i in aln_masked]
for pos, minor in zip(homoplasy_dominant_clust_enrich_top.Position, homoplasy_dominant_clust_enrich_top.minor):
    snp_position=snp_pos_dict[str(pos)]
    minor_allele_list=[1 if i==minor else 0 for i in list(aln_masked[:,snp_position])]
    allele_dict[pos+'_'+minor]=minor_allele_list
    gene_list.append(pos_gene_dict[str(pos)]['Gene'])

In [8]:
dominant_clust_snp_table=pd.DataFrame.from_dict(allele_dict).iloc[:,0:51].set_index('name')
dominant_clust_snp_table

,4770807_A,4211627_T,1217813_T,3381893_G,2119889_T,3279849_A,3280500_A,3280832_A,3520983_T,3016615_A,...,4103062_C,4103870_T,4103936_C,728418_T,1055261_C,1285144_T,3254305_C,29537_A,3069071_T,3069251_C
name,,,,,,,,,,,,,,,,,,,,,
SAMEA1317694,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
SAMEA1464939,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
SAMEA1464888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SAMEA1464902,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
SAMEA1464890,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMEA5396772,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
SAMEA5396773,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
SAMEA5396791,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [9]:
dominant_clust_snp_table.columns=gene_list
dominant_clust_snp_table

,MAB_4690c,MAB_4147c,MAB_1203,MAB_3334c,MAB_2120c,MAB_3243,MAB_3244,MAB_3244,MAB_3480,MAB_2961,...,MAB_4055c,MAB_4056c,MAB_4056c,MAB_0724c,MAB_1046c,MAB_1279,MAB_3213c,MAB_0032,MAB_3010,MAB_3010
name,,,,,,,,,,,,,,,,,,,,,
SAMEA1317694,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
SAMEA1464939,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
SAMEA1464888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SAMEA1464902,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
SAMEA1464890,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMEA5396772,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
SAMEA5396773,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
SAMEA5396791,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [10]:
dominant_clust_snp_table.to_csv('/n/data1/hms/dbmi/farhat/nikki/abscessus/0_NOTEBOOKS/010_homoplasy/data/20210902_dominant_cluster_enrichment_snp_table.csv', sep=',')

In [11]:
!head -500 /n/data1/hms/dbmi/farhat/nikki/abscessus/0_NOTEBOOKS/010_homoplasy/data/20210902_dominant_cluster_enrichment_snp_table.csv

name,MAB_4690c,MAB_4147c,MAB_1203,MAB_3334c,MAB_2120c,MAB_3243,MAB_3244,MAB_3244,MAB_3480,MAB_2961,MAB_1251c,MAB_4141,MAB_4141,-,MAB_4141,MAB_4141,MAB_4145,MAB_4557c,MAB_3242,MAB_3481,MAB_0958c,MAB_4141,MAB_0834,MAB_4057c,MAB_2962,MAB_0033c,MAB_3094c,MAB_3581c,MAB_4469c,MAB_2495,MAB_2962,MAB_4057c,MAB_1054,MAB_1054,MAB_3515c,MAB_0519,MAB_0519,MAB_1048c,MAB_0826c,MAB_3507,MAB_4055c,MAB_4056c,MAB_4056c,MAB_0724c,MAB_1046c,MAB_1279,MAB_3213c,MAB_0032,MAB_3010,MAB_3010
SAMEA1317694,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
SAMEA1464939,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
SAMEA1464888,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SAMEA1464902,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
SAMEA1464890,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1